## STEP 1

Import Libraries

In [38]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.plotting as plot
import math


Create the Medium Voltage net

In [17]:
net = nw.create_cigre_network_mv(with_der=False)

Run the net

In [ ]:
net

# 1.1 

What is the state of the network? Check bus voltage [pu] and line/transformer loading [%]

Consider that the maximum allowed voltage is 1.1 pu, the minimum is 0.9 pu, and the maximum line loading is 100%.

Set the limits for Voltage Per Unit (pu) and for Line loading

In [30]:
bus_max_pu=1.1
bus_min_pu=0.9
max_line_loading=100
max_trafo_loading=100

Run the net

In [ ]:
pp.runpp(net)
plot.simple_plot(net, respect_switches=False, line_width=1.0, bus_size=1.0, ext_grid_size=1.0, trafo_size=1.0, 
                 plot_loads=True, plot_gens=False, plot_sgens=False, 
                 load_size=2, gen_size=1.0, sgen_size=1.0, 
                 switch_size=2.0, switch_distance=1.0, plot_line_switches=False, scale_size=True, 
                 bus_color='b', line_color='grey', trafo_color='k', ext_grid_color='y', switch_color='r', library='igraph', show_plot=True, ax=None)


Define a function to Check if Voltage is within the limits max_pu and min_pu

In [48]:
def checkVoltageRange(max,min,vm_data,component_name):
    print(f'---------{component_name} Voltage Check ---------\n')
    i=0
    faults=0
    for vm in vm_data:
        if vm >= max:
            print(f'{component_name} {i} Voltage is Over the Limit: {round(vm,4)}')
            faults+=1
        elif vm <= min:
            print (f'{component_name} {i} Voltage is Under the Limit: {round(vm,4)}')
            faults+=1
        elif math.isnan(vm):
            print (f'{component_name} {i} is off Grid')
            faults+=1
        i+=1
    if faults==0:
        print(f'OK {component_name} \n')
    else:
        print(f'Number of {component_name} out of Voltage range: {faults}\n ')

Define a function to check if Loading Percent is over a certain Limit

In [47]:
def checkLoadingPercent(limit,loading_data,component_name):
    print(f'---------{component_name} Loading Check ---------\n')
    i=0
    faults=0
    for loading in loading_data:
        if loading > limit:
            print(f'{component_name}  {i} is Overloaded: {round(loading,2)}')
            faults+=1
        elif math.isnan(loading):
            print (f'{component_name} {i} is off Grid')
            faults+=1
        i+=1
    if faults==0:
        print(f'OK {component_name}\n')
    else:
        print(f'Number of Overloaded {component_name} : {faults} \n')
        

Check the requried Network values

In [ ]:
checkVoltageRange(bus_max_pu,bus_min_pu,net.res_bus.vm_pu,"Bus")
checkLoadingPercent(max_line_loading,net.res_line.loading_percent,"Line")
checkLoadingPercent(max_trafo_loading,net.res_trafo.loading_percent,"Transformer")

# 1.2

What happens in the system during a contingency (N-1) case? Are there lines overloaded? Buses above/under the limits?

Consider that the maximum allowed voltage is 1.1 pu, the minimum is 0.9 pu, the maximum line loading is 100% AND all
customers/loads should be supplied with power.

Ignore the switches:
- Line 12 (bus 6 to 7)
- Line 13 (bus 11 to 4)
- Line 14 (bus 14 to 8)

Define a function to check if Load receive power

In [24]:
def load_check(load_data):
    print(f'----------Loads check ---------\n')
    i=0
    faults=0
    for load in load_data:
        if load == 0:
            #print(f'{i} OK')
            faults+=1
        i+=1
    if faults==0:
        print("OK Loads\n")
    else:
        print(f'Number of not served Loads: {faults}\n')

Run the N-1 contingency with the normal limit values

In [ ]:

i=0
for i,line in net.line.iterrows():
    if i < 12:  
        #Printing Line Index and From-To Buses   
        print(f'######################## Line {i} ##############################')
        from_bus = line['from_bus']
        to_bus = line['to_bus']
        print(f'From Bus {from_bus} to {to_bus}')

        #Line i Out of service 
        net.line.at[i,'in_service']=False

        #Run the net
        pp.runpp(net)

        #Check the values
        checkVoltageRange(bus_max_pu,bus_min_pu,net.res_bus.vm_pu,"Bus")
        checkLoadingPercent(max_line_loading,net.res_line.loading_percent,"Line")
        checkLoadingPercent(max_trafo_loading,net.res_trafo.loading_percent,"Transformer")
        print(net.res_trafo.loading_percent)
        print(net.res_trafo.p_hv_mw)
        
        load_check(net.res_load.p_mw)
        
        net.line.at[i,'in_service']=True
        i+=1

# 1.3

Run the N-1 contingency with the **Strict** limit values

In [ ]:
i=0

bus_max_pu=1.05
bus_min_pu=0.95


for i,line in net.line.iterrows():
    if i < 12:  
        #Printing Line Index and From-To Buses   
        print(f'######################## Line {i} ##############################')
        from_bus = line['from_bus']
        to_bus = line['to_bus']
        print(f'From Bus {from_bus} to {to_bus}')

        #Line i Out of service 
        net.line.at[i,'in_service']=False

        #Run the net
        pp.runpp(net)

        #Check the values
        checkVoltageRange(bus_max_pu,bus_min_pu,net.res_bus.vm_pu,"Bus")
        net.res_bus.
        checkLoadingPercent(max_line_loading,net.res_line.loading_percent,"Line")
        checkLoadingPercent(max_trafo_loading,net.res_trafo.loading_percent,"Transformer")
        load_check(net.res_load.p_mw)
        
        net.line.at[i,'in_service']=True
        i+=1

## STEP 2

Add load shapes to existing customers (time series simulation)